# Setting Up Our Environment

___

In [2]:
import alpaca_trade_api as tradeapi
import pandas as pd
from tabulate import tabulate
import time
from datetime import datetime, timedelta


import os
from dotenv import load_dotenv, find_dotenv
from pathlib import Path

# Obtaining Credentials

In order to use the Stock Trader each user must have a API & SECRET key in order to abide by Alpaca's documentation

In [3]:
def get_credentials():
    load_dotenv("creds.env")
    api_key = os.getenv("ALPACA_API_KEY")
    secret_key = os.getenv("SECRET_KEY")
    if not api_key and not secret_key:
        print("API key not found. Please check your creds.env file.")
    else:
        api = tradeapi.REST(api_key, secret_key, base_url='https://paper-api.alpaca.markets')
        print("API key loaded successfully.")
    return api

api = get_credentials()

API key loaded successfully.


## Placing Orders

### Places an order based on the Stock
- symbol (str) : The name of the Stock
- qty : Amount of shares to buy or sell
- order_type : Order Type
    - Market by default
- side : Buy or Sell
    - default is Buy

In [4]:
def place_order(symbol, qty, order_type="market", side="buy"):

    try:
        api.submit_order(
            symbol=symbol,
            qty=qty,
            side=side,
            type=order_type,
            time_in_force='gtc'
        )
        print(f"Order placed: {side} {qty} shares of {symbol}")
    except Exception as e:
        print(f"Error placing order: {e}")




# Simple Moving Average

### Returns the simple moving average for the given stock
- data : stock data
- window : the size for the moving average (possibly adjust)

### Return:
    - Pandas Data Frame of Moving Averages

In [5]:
def calculate_SMA(data, window):
    return data['Close'].rolling(window=window).mean()

# Calculating Strength
- data : Stock Data
- window : the size for the moving strength index

### Returns:
    - Pandas Data Frame of the RSI Values



In [6]:
def calculate_RSI(data, window=14):
    delta = data['Close'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    RS = gain / loss
    RSI = 100 - (100 / (1 + RS))
    return RSI

# Fetching Data


In [24]:
def fetch_data(stock_symbol, period='5y', interval='1m', live=False):
    try:
        if live:
            # Fetch live data (Bugs here)
            bars = api.get_bars(stock_symbol, timeframe=interval, limit=1).df
            if bars.empty:
                print(f"No live data available for {stock_symbol}.")
                return None
            return bars
        else:
            # Calculate the correct dates
            if period.endswith('d'):
                num_days = int(period[:-1])  # Extract the number of days
                end_date = datetime.now()
                start_date = end_date - timedelta(days=num_days)
            else:
                print("Unsupported format. Please specify days ex: '10d'")
                return None

            # Convert dates to proper format
            start_iso = start_date.isoformat() + 'UTC'  # 
            end_iso = end_date.isoformat() + 'UTC'

            print(f"Fetching historical data for {stock_symbol} from {start_iso} to {end_iso}...")
            bars = api.get_bars(stock_symbol, timeframe=interval, start=start_iso, end=end_iso).df
            if bars.empty:
                print(f"No historical data available for {stock_symbol} in the given period.")
                return None
            print(f"Data retrieved successfully for {stock_symbol}.")
            return bars
    except Exception as e:
        print(f"An error occurred while fetching data for {stock_symbol}: {e}")
        return None

# Example usage
bars = fetch_data('TSLA', period='10d', interval='5Min', live=False)
print(bars)


Fetching historical data for TSLA from 2024-08-08T18:01:25.757587Z to 2024-08-18T18:01:25.757587Z...
Data retrieved successfully for TSLA.
                              close      high       low  trade_count  \
timestamp                                                              
2024-08-08 18:05:00+00:00  199.8379  200.0700  199.5704         8515   
2024-08-08 18:10:00+00:00  200.2040  200.2500  199.6600         5413   
2024-08-08 18:15:00+00:00  200.6134  200.6800  200.0870         8693   
2024-08-08 18:20:00+00:00  200.4400  200.7000  200.3900         5821   
2024-08-08 18:25:00+00:00  200.0373  200.5167  199.9000         6323   
...                             ...       ...       ...          ...   
2024-08-16 23:35:00+00:00  216.2400  216.2900  216.2011           92   
2024-08-16 23:40:00+00:00  216.2700  216.2700  216.2008          128   
2024-08-16 23:45:00+00:00  216.3000  216.3000  216.2100          116   
2024-08-16 23:50:00+00:00  216.3200  216.3200  216.2407          129 

# Trading Strategy

In [8]:
def trading_strategy(stock_symbol, qty):
    # Fetch the latest data
    data = fetch_data(stock_symbol, period='10d', interval='5Min', live=True)
    
    if data is not None and not data.empty:
        # Calculate SMA and RSI
        sma = calculate_SMA(data)
        rsi = calculate_RSI(data)
        
        # Get the latest close price
        latest_close = data['close'].iloc[-1]
        latest_sma = sma.iloc[-1]
        latest_rsi = rsi.iloc[-1]

        # Setting the trading conditions (Fix Bugs here)
        if latest_rsi < 30 and latest_close < latest_sma:
            # Buy signal (Update my logic here)
            place_order(stock_symbol, qty, side='buy')
        elif latest_rsi > 70 and latest_close > latest_sma:
            # Sell signal
            place_order(stock_symbol, qty, side='sell')
    else:
        print(f"Failed to retrieve data for {stock_symbol}")

# Running the Trading Bot

In [ ]:
def run_bot(stock_symbol, qty, interval=60):
    while True:
        trading_strategy(stock_symbol, qty)
        time.sleep(interval)  # Wait for the next iteration

# To De-Bug
run_bot('AAPL', 10, interval=300)  